In [23]:
import os
import time
import pandas as pd
import simplekml
import shutil
from sys import platform
from pathlib import Path

# For Vincenty Formula
from math import atan
from math import atan2
from math import cos
from math import radians
from math import sin
from math import sqrt
from math import tan

pd.options.mode.chained_assignment = None

# Measuring execution time
start_time = time.time()

# Specify Root Path

In [24]:
# Checks what operating system the user is using
if platform == 'win32':
    root_path = r'C:\Users\Stefanos\DeLasCasas CP, LLc\Ro De Las Casas - ' \
                r'DLC2P Server\AA Stefanos\Data Science\Python Projects\CIS ' \
                r'Data to Google Earth (Real Time)'
elif platform == 'darwin':
    root_path = r'/Users/Stefanos/Library/CloudStorage/' \
                r'OneDrive-SharedLibraries-DeLasCasasCP,LLc/Ro De Las Casas ' \
                r'- DLC2P Server/AA Stefanos/Data Science/Python Projects/' \
                r'CIS Data to Google Earth (Real Time)'

# Specify Variables

In [25]:
# SPECIFY ICONS
icon_on = 'https://img.icons8.com/material-sharp/24/1ff08a/give-way--v1.png'
icon_off = 'https://img.icons8.com/material-sharp/24/7950F2/give-way--v1.png'
icon_comments = 'https://img.icons8.com/ultraviolet/80/000000/comments.png'
icon_1200 = 'https://img.icons8.com/ios-filled/50/22C3E6/360-degrees.png'
icon_850 = 'https://img.icons8.com/ios-filled/50/F25081/360-degrees.png'
# SPECIFY LINE COLORS
# color_1200 = simplekml.Color.rgb(18, 201, 232, 255)
# color_850 = simplekml.Color.rgb(240, 31, 93, 255)
# SPECIFY SCALE FACTOR
scale_factor = 175
# SPECIFY TRANSPARENCY (0 to 1)
transparency = 1
# SPECIFY ICON SCALE
icon_scale = 0.2
# SPECIFY COLOR PALETTE
#my_cmap = plt.get_cmap('Spectral_r')
# PROJECT NAME
project_name = 'XYZ'    #input('\nPROJECT NAME:')

# Vincenty Formula

In [26]:
class VincentyInverse:
    def __init__(self, coord1, coord2, maxiter=200, tol=10**-12):

        #--- CONSTANTS ------------------------------------+

        a = 6378137.0                    # radius at equator in meters (WGS-84)
        f = 1/298.257223563              # flattening of the ellipsoid (WGS-84)
        b = (1-f)*a

        phi_1, lon_1, = coord1                      # (lat=L_?,lon=phi_?)
        phi_2, lon_2, = coord2

        u_1 = atan((1-f)*tan(radians(phi_1)))
        u_2 = atan((1-f)*tan(radians(phi_2)))

        L = radians(lon_2-lon_1)

        Lambda = L                           # set initial value of lambda to L

        sin_u1 = sin(u_1)
        cos_u1 = cos(u_1)
        sin_u2 = sin(u_2)
        cos_u2 = cos(u_2)

        #--- BEGIN ITERATIONS -----------------------------+
        self.iters = 0
        for i in range(0, maxiter):
            self.iters += 1

            cos_lambda = cos(Lambda)
            sin_lambda = sin(Lambda)
            sin_sigma = sqrt((cos_u2*sin(Lambda))**2+
                             (cos_u1*sin_u2-sin_u1*cos_u2*cos_lambda)**2)
            cos_sigma = sin_u1*sin_u2+cos_u1*cos_u2*cos_lambda
            sigma = atan2(sin_sigma,cos_sigma)
            sin_alpha = (cos_u1*cos_u2*sin_lambda)/sin_sigma
            cos_sq_alpha = 1-sin_alpha**2
            cos2_sigma_m = cos_sigma-((2*sin_u1*sin_u2)/cos_sq_alpha)
            C = (f/16)*cos_sq_alpha*(4+f*(4-3*cos_sq_alpha))
            Lambda_prev = Lambda
            Lambda = L+(1-C)*f*sin_alpha*(sigma+C*sin_sigma*
                                          (cos2_sigma_m+C*cos_sigma*
                                           (-1+2*cos2_sigma_m**2)))

            # Successful convergence
            diff = abs(Lambda_prev-Lambda)
            if diff <= tol:
                break

        u_sq = cos_sq_alpha*((a**2-b**2)/b**2)
        A = 1+(u_sq/16384)*(4096+u_sq*(-768+u_sq*(320-175*u_sq)))
        B = (u_sq/1024)*(256+u_sq*(-128+u_sq*(74-47*u_sq)))
        delta_sig = B*sin_sigma*(cos2_sigma_m+0.25*B*
                                 (cos_sigma*(-1+2*cos2_sigma_m**2)-
                                  (1/6)*B*cos2_sigma_m*(-3+4*sin_sigma**2)*
                                  (-3+4*cos2_sigma_m**2)))

        self.m = b*A*(sigma-delta_sig)         # output distance in meters
        #self.km = self.m/1000                 # output distance in kilometers
        #self.mm = self.m*1000                 # output distance in millimeters
        self.miles = self.m*0.000621371        # output distance in miles
        self.ft = self.miles*5280              # output distance in feet
        #self.inches = self.ft*12              # output distance in inches
        #self.yards = self.ft/3                # output distance in yards

# if __name__ == "__VincentyInverse__":
#     main()

# Data Cleaning

In [27]:
# Creates a list of files in data folder

# Checks what operating system the user is using
if platform == 'win32':
    raw_data_list = os.listdir(r'/Users/Stefanos/'
                           r'DeLasCasas CP, LLc/'
                           r'Ro De Las Casas - DLC2P Server/AA Stefanos/'
                           r'Data Science/Python Projects/'
                           r'CIS Data to Google Earth (Real Time)/'
                           r'Original Data')
elif platform == 'darwin':
    raw_data_list = os.listdir(r'/Users/Stefanos/Library/CloudStorage/'
                           r'OneDrive-SharedLibraries-DeLasCasasCP,LLc/'
                           r'Ro De Las Casas - DLC2P Server/AA Stefanos/'
                           r'Data Science/Python Projects/'
                           r'CIS Data to Google Earth (Real Time)/'
                           r'Original Data')

In [28]:
# Create an empty list to exclude certain files
filtered_data_list = []
# Current list index
i = 0

# Filters the folder and excludes files that start with '.'
while i < len(raw_data_list):
    if not raw_data_list[i][0] == '.':
        filtered_data_list.append(raw_data_list[i])
    i += 1

In [29]:
# Creates an empty list to place the total miles
total_miles_list = []
# Current list index
i = 0
j = 0

# Goes through all the files in the list
while j < len(filtered_data_list):
    # Create dataframe from a csv file
    df_cis = pd.read_csv(root_path +
                         str(Path(r'/Original Data/' +
                                  filtered_data_list[j])))
    # Drops elevation and PDOP columns
    df_cis = df_cis.iloc[:, :6]
    # Delete rows that CIS was skipped
    df_cis = df_cis.loc[df_cis['On Potential'] != 'SKIP'].reset_index(drop=True)
    # Convert object columns to numbers
    df_cis[['On Potential', 'Off Potential']] = \
                            df_cis[['On Potential', 'Off Potential']]\
                            .apply(pd.to_numeric, errors='coerce', axis=1)
    df_cis[['On Potential', 'Off Potential']] /= 1000
    # Trim white space
    df_cis['Comments'] = df_cis['Comments'].str.strip()
    df_cis['Distance (ft)'] = 0
    last_index = df_cis.last_valid_index()
    # Calculates distance between GPS coordinates
    while i < last_index:
        try:
            df_cis['Distance (ft)'][i+1] = \
            VincentyInverse([df_cis['Latitude'][i],
                             df_cis['Longitude'][i]],
                            [df_cis['Latitude'][i+1],
                             df_cis['Longitude'][i+1]]).ft
        except ZeroDivisionError:
            df_cis['Distance (ft)'][i+1] = 0
        i += 1
    # Records total miles in a list
    total_miles_list.append(round(max(df_cis['Station'])/5280, 2))
    df_cis.to_csv(root_path +
                  str(Path(r'/Data/' +
                           filtered_data_list[j])), index=False)
    i = 0
    j += 1

# Create Google Earth Folders

In [30]:
# Replaces output folder with an empty one
if os.path.exists(root_path + r'/Output/'):
    shutil.rmtree(root_path + r'/Output/')
    os.makedirs(root_path + r'/Output/')
if not os.path.exists(root_path + r'/Output/'):
    os.makedirs(root_path + r'/Output/')

In [31]:
cis_kml =[]
type_folders = []
current_mile = 1
k = 0
j = 0       # List counter for CIS Survey '.kml' files
i = 0       # List counter for mile folders

In [32]:
# Create CIS survey '.kml' files per mile #
while j < len(total_miles_list):
    miles_remaining = total_miles_list[j]
    # Create type folders
    while miles_remaining > 0:
        cis_kml.append(filtered_data_list[j].split('.DAT', 1)[0] +
                       ' (Mile ' + current_mile.__str__() + ')')
        cis_kml[i] = simplekml.Kml()
        type_folders.append(filtered_data_list[j].split('.DAT', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(On)')
        type_folders.append(filtered_data_list[j].split('.DAT', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(Off)')
        type_folders.append(filtered_data_list[j].split('.DAT', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(Comments)')
        type_folders.append(filtered_data_list[j].split('.DAT', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(-1.2 V)')
        type_folders.append(filtered_data_list[j].split('.DAT', 1)[0] +
                            ' (Mile ' + current_mile.__str__() + ')(-0.85 V)')
        type_folders[k] = cis_kml[i].newfolder(name='On')
        type_folders[k+1] = cis_kml[i].newfolder(name='Off')
        type_folders[k+2] = cis_kml[i].newfolder(name='Comments')
        type_folders[k+3] = cis_kml[i].newfolder(name='-1.2 V')
        type_folders[k+4] = cis_kml[i].newfolder(name='-0.85 V')
        cis_kml[i].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.DAT', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = total_miles_list[j]
        i += 1
        k += 5
    j += 1
    current_mile = 1

In [33]:
# cis_kml =[]
# mile_folders = []
# folder_mile = 1
# j = 0       # List counter for CIS Survey '.kml' files
# i = 0       # List counter for mile folders
#
# # Create CIS survey '.kml' files
# while j < len(total_miles_list):
#     miles_remaining = total_miles_list[j]
#     cis_kml.append(filtered_data_list[j].split('.DAT', 1)[0])
#     cis_kml[j] = simplekml.Kml()
#     # Create mile # folders
#     while miles_remaining > 0:
#         mile_folders.append('Mile ' + folder_mile.__str__())
#         mile_folders[i] = cis_kml[j].newfolder(name=mile_folders[i])
#         folder_mile += 1
#         miles_remaining -= 1
#         i += 1
#         if 0 < miles_remaining < 1:
#             folder_mile = total_miles_list[j]
#         cis_kml[j].savekmz(root_path + r'/Output/' +
#                           filtered_data_list[j].split('.DAT', 1)[0] + '.kmz')
#     j += 1
#     folder_mile = 1

In [34]:
#df = df_cis_on[df_cis_on.duplicated(subset=['Longitude', 'Latitude'],
                                    #keep=False)]

# CIS Data

## On

In [35]:
feet_counter = 5280
style = simplekml.Style()
i = 0       # Row for data points
j = 0       # Row for CIS surveys
k = 0       # Counter for type folders (0 = 'On')
kml_file = 0

# Goes through all CIS survey files
while j < len(filtered_data_list):
    df_cis_on = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_on = df_cis_on[df_cis_on['On Potential'] != 0]
    df_cis_on = df_cis_on[['Station', 'Longitude', 'Latitude', 'On Potential']]\
        .reset_index(drop=True)
    df_cis_on['On Potential'] = df_cis_on['On Potential']*(-1)
    last_index = df_cis_on.last_valid_index()
    miles_remaining = total_miles_list[j]
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_on.loc[i, 'Station'] < feet_counter:
            pnt = type_folders[k]\
                .newpoint(name=df_cis_on.loc[i, 'On Potential']*(-1),
                          visibility=0)
            pnt.style.balloonstyle.text = \
                'Potential: -{} V\nLongitude: {}\nLatitude: {}\nStation: {} ft'\
                    .format(df_cis_on.loc[i, 'On Potential'],
                            df_cis_on.loc[i, 'Longitude'],
                            df_cis_on.loc[i, 'Latitude'],
                            df_cis_on.loc[i, 'Station'])
            pnt.coords = [(df_cis_on.loc[i, 'Longitude'],
                           df_cis_on.loc[i, 'Latitude'],
                           df_cis_on.loc[i, 'On Potential']*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_on
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.DAT', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = total_miles_list[j]
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

## Off

In [36]:
k = 1       # Counter for type folders (1 = 'Off')

# Goes through all CIS survey files
while j < len(filtered_data_list):
    df_cis_off = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_off = df_cis_off[df_cis_off['Off Potential'] != 0]
    df_cis_off = df_cis_off[['Station', 'Longitude', 'Latitude',
                           'Off Potential']].reset_index(drop=True)
    df_cis_off['Off Potential'] = df_cis_off['Off Potential']*(-1)
    last_index = df_cis_off.last_valid_index()
    miles_remaining = total_miles_list[j]
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_off.loc[i, 'Station'] < feet_counter:
            pnt = type_folders[k]\
                .newpoint(name=df_cis_off.loc[i, 'Off Potential']*(-1),
                          visibility=0)
            pnt.style.balloonstyle.text = \
                'Potential: -{} V\nLongitude: {}\nLatitude: {}\nStation: {} ft'\
                    .format(df_cis_off.loc[i, 'Off Potential'],
                            df_cis_off.loc[i, 'Longitude'],
                            df_cis_off.loc[i, 'Latitude'],
                            df_cis_off.loc[i, 'Station'])
            pnt.coords = [(df_cis_off.loc[i, 'Longitude'],
                           df_cis_off.loc[i, 'Latitude'],
                           df_cis_off.loc[i, 'Off Potential']*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_off
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.DAT', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = total_miles_list[j]
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

## Comments

In [37]:
k = 2       # Counter for type folders (2 = 'Comments')

# Goes through all CIS survey files
while j < len(filtered_data_list):
    df_cis_comments = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    df_cis_comments.dropna(inplace=True)
    df_cis_comments.reset_index(drop=True, inplace=True)
    last_index = df_cis_comments.last_valid_index()
    miles_remaining = total_miles_list[j]
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_comments.loc[i, 'Station'] < \
                feet_counter:
            pnt = type_folders[k]\
                .newpoint(name=df_cis_comments.loc[i,'Comments'], visibility=0)
            if df_cis_comments.loc[i, 'On Potential'] == 0:
                potential = df_cis_comments.loc[i, 'Off Potential'].__str__()\
                            + ' V (Off)'
            else:
                potential = df_cis_comments.loc[i, 'On Potential'].__str__() \
                            + ' V (On)'
            pnt.style.balloonstyle.text = \
                'Potential: {}\nLongitude: {}\nLatitude: {}\n' \
                'Station: {} ft\nComment: {}'\
                    .format(potential,
                            df_cis_comments.loc[i, 'Longitude'],
                            df_cis_comments.loc[i, 'Latitude'],
                            df_cis_comments.loc[i, 'Station'],
                            df_cis_comments.loc[i, 'Comments'])
            pnt.coords = [(df_cis_comments.loc[i, 'Longitude'],
                           df_cis_comments.loc[i, 'Latitude'],
                           0)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_comments
            pnt.style.iconstyle.scale = icon_scale*1.5
            pnt.style.labelstyle.scale = 0.5
            #pnt.extrude = 0
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.DAT', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = total_miles_list[j]
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

## -1.2 V

In [38]:
k = 3       # Counter for type folders (3 = '-1.2 V')

# Goes through all CIS survey files
while j < len(filtered_data_list):
    df_cis_1200 = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    last_index = df_cis_1200.last_valid_index()
    miles_remaining = total_miles_list[j]
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_1200.loc[i, 'Station'] < \
                feet_counter:
            pnt = type_folders[k].newpoint(name='-1.200', visibility=0)
            pnt.style.balloonstyle.text = 'Potential: -1.2 V'
            pnt.coords = [(df_cis_1200.loc[i, 'Longitude'],
                           df_cis_1200.loc[i, 'Latitude'],
                           1.2*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_1200
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.DAT', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = total_miles_list[j]
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

## -0.85 V

In [39]:
k = 4       # Counter for type folders (4 = '-0.85 V')

# Goes through all CIS survey files
while j < len(filtered_data_list):
    df_cis_850 = pd.read_csv(root_path + r'/Data/' + filtered_data_list[j])
    last_index = df_cis_850.last_valid_index()
    miles_remaining = total_miles_list[j]
    # Create '.kml' files for each mile #
    while miles_remaining > 0:
        # Create 3D data points
        while i <= last_index and df_cis_850.loc[i, 'Station'] < \
                feet_counter:
            pnt = type_folders[k].newpoint(name='-0.850', visibility=0)
            pnt.style.balloonstyle.text = 'Potential: -0.85 V'
            pnt.coords = [(df_cis_850.loc[i, 'Longitude'],
                           df_cis_850.loc[i, 'Latitude'],
                           0.85*scale_factor)]
            pnt.altitudemode = simplekml.AltitudeMode.relativetoground
            pnt.style.iconstyle.icon.href = icon_850
            pnt.style.iconstyle.scale = icon_scale
            pnt.style.labelstyle.scale = 0
            pnt.extrude = 0
            i += 1
        cis_kml[kml_file].savekmz(root_path + r'/Output/' +
                           filtered_data_list[j].split('.DAT', 1)[0] +
                           ' (Mile ' + current_mile.__str__() + ').kmz')
        current_mile += 1
        miles_remaining -= 1
        if 0 < miles_remaining < 1:
            current_mile = total_miles_list[j]
        feet_counter += 5280
        k += 5
        kml_file += 1
    # Reset variables for next CIS survey
    feet_counter = 5280
    current_mile = 1
    i = 0
    j += 1
j = 0
kml_file = 0

In [40]:
#print('Execution Time: %s seconds' % round(time.time() - start_time))

In [41]:
round(time.time() - start_time)/60

2.6666666666666665

## Pipe

In [42]:
# kml = simplekml.Kml()

In [43]:
# df_cis_pipe = df_cis.iloc[:, [False, False, False, False, True, True, False]].copy()
# df_cis_pipe['Potential (V)'] = 0

In [44]:
# z = 282                                  # Max iterations
#
# while i < z:
#     ls = kml.newlinestring()
#     ls.style.balloonstyle.text = 'Ground'
#     ls.coords = [(df_cis_pipe.loc[i, 'LONGITUDE'], df_cis_pipe.loc[i, 'LATITUDE'], df_cis_pipe.loc[i, 'Potential (V)']),
#                  (df_cis_pipe.loc[i+1, 'LONGITUDE'], df_cis_pipe.loc[i+1, 'LATITUDE'], df_cis_pipe.loc[i+1, 'Potential (V)'])]
#
#     # Style
#     ls.style.linestyle.width = 5
#     ls.style.linestyle.color = simplekml.Color.rgb(255, 255, 255, 175)
#
#     i += 1
#
# kml.save(root_path + r'\Output/' + 'Ground - ' + project_name + '.kml')